介绍层和块(layers,block)

块（block）可以描述单个层、由多个层组成的组件或整个模型本身。  
使用块进行抽象的一个好处是可以将一些块组合成更大的组件， 这一过程通常是递归的

从编程的角度来看，块由类（class）表示。  
它的任何子类都必须定义一个将其输入转换为输出的前向传播函数， 并且必须存储任何必需的参数。  
注意，有些块不需要任何参数。 最后，为了计算梯度，块必须具有反向传播函数。  
在定义我们自己的块时，由于自动微分（在 2.5节 中引入） 提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层， 然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)


tensor([[ 0.1822,  0.2442, -0.1306,  0.1434, -0.1137,  0.1714, -0.2374,  0.0860,
         -0.1158,  0.0176],
        [ 0.1137,  0.0519, -0.0602,  0.1616,  0.0777,  0.0867, -0.1549,  0.0853,
         -0.0359, -0.0367]], grad_fn=<AddmmBackward0>)

通过实例化nn.Sequential来构建我们的模型， 层的执行顺序是作为参数传递的。  
简而言之，nn.Sequential定义了一种特殊的Module， 即在PyTorch中表示一个块的类， 它维护了一个由Module组成的有序列表。  
注意，两个全连接层都是Linear类的实例， Linear类本身就是Module的子类。  
另外，到目前为止，我们一直在通过net(X)调用我们的模型来获得模型的输出。  
这实际上是net.\__call\__(X)的简写。  
这个前向传播函数非常简单： 它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。

<pre>自定义块  
简要总结一下每个块必须提供的基本功能  

    1.将输入数据作为其前向传播函数的参数。

    2.通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

    3.计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

    4.存储和访问前向传播计算所需的参数。

    5.根据需要初始化模型参数。



从零开始编写一个块。  
它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。  
注意，下面的MLP类继承了表示块的类。  
我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

In [4]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self,X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

除非我们实现一个新的运算符， 否则我们不必担心反向传播函数或参数初始化， 系统将自动生成这些

In [5]:
net = MLP()
net(X)


tensor([[ 0.1033, -0.1391,  0.0533, -0.0606, -0.0818,  0.0706,  0.0736,  0.0912,
         -0.0832, -0.0205],
        [ 0.0925, -0.1671,  0.0274, -0.0266, -0.0350, -0.0092,  0.1303,  0.1412,
         -0.0008,  0.0727]], grad_fn=<AddmmBackward0>)

顺序块  
看看Sequential类是如何工作的    
构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：  

    一种将块逐个追加到列表中的函数；  

    一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。  

下面的MySequential类提供了与默认Sequential类相同的功能

In [18]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)]=module

    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X

__init__函数将每个模块逐个添加到有序字典_modules中。  
读者可能会好奇为什么每个Module都有一个_modules属性？   
以及为什么我们使用它而不是自己定义一个Python列表？   
简而言之，_modules的主要优点是：   
在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。

当MySequential的前向传播函数被调用时， 每个添加的块都按照它们被添加的顺序执行。   

现在可以使用我们的MySequential类重新实现多层感知机。

In [19]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)


tensor([[ 0.1562,  0.0602,  0.0727,  0.2359, -0.1304,  0.2054, -0.2025,  0.0031,
         -0.1251,  0.1205],
        [ 0.1281,  0.1897, -0.0363, -0.0802, -0.1686,  0.0846, -0.0511,  0.0547,
         -0.0493,  0.2548]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码  
Sequential类使模型构造变得简单， 允许我们组合新的架构，而不必定义自己的类。   
然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。   
例如，我们可能希望在前向传播函数中执行Python的控制流。   
此外，我们可能希望执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

有时我们可能希望合并既不是上一层的结果也不是可更新参数的项， 我们称之为常数参数（constant parameter）。  
例如，我们需要一个计算函数$f(x,w)=c \cdot w^Tx$的层， 其中x是输入， w是参数， c是某个在优化过程中没有更新的指定常量。

In [25]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数C。因此其在训练期间保持不变
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    def forward(self,X):
        X=self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X=self.linear(X)
        # 控制流
        while X.abs().sum()>1:
            X/=2
        return X.sum()


实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。  
这个权重不是一个模型参数，因此它永远不会被反向传播更新。  
然后，神经网络将这个固定层的输出通过一个全连接层  
模型做了一些不寻常的事情:  
行了一个while循环，在L1范数大于1的条件下， 将输出向量除以2，直到它满足条件为止。  
最后，模型返回了X中所有项的和。  
注意，此操作可能不会常用于在任何实际任务中， 我们只展示如何将任意代码集成到神经网络计算的流程中。

In [26]:
net = FixedHiddenMLP()
net(X)


tensor(-0.0002, grad_fn=<SumBackward0>)

可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。

In [30]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(20,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
        )
        self.linear=nn.Linear(32,16)
    
    def forward(self,X):
        return self.linear(self.net(X))

chimera=nn.Sequential(
    NestMLP(),
    nn.Linear(16,20),
    FixedHiddenMLP())

chimera(X)

tensor(0.4044, grad_fn=<SumBackward0>)